# Ex. 3: Creating e-mail alerts for changing fire weather conditions

Now that we've explored using the data in some elementary visualization examples, let's consider a script that allows us to send e-mail alerts when some stations meet specific conditions. For this example, I'll send an alert e-mail when winds are above 15 mph and relative humidity is less than 30% at stations in the Southern GACC in the past hour. 

##### Libraries Used:
- smtplib
- urllib
- json
- email
- time

### Constructing our query looks like:

http://api.mesowest.net/v2/stations/nearesttime?&token=yo&gacc=sacc&within=60&vars=relative_humidity,wind_speed&units=english&timeformat=%H:%M&obtimezone=local

Where we are specifically targetting the Southern GACC RAWS within the last 60 minutes for only the relative humidity and wind speed variables in English units. 

### Like before, our API calling block is:

In [74]:
import urllib.request
import json

q = 'http://api.mesowest.net/v2/stations/nearesttime?&token=yo&gacc=sacc&within=60&vars=relative_humidity,wind_speed&units=english&timeformat=%H:%M&obtimezone=local'
# Open the url and read the contents
resp = urllib.request.urlopen(q).read()
# Decode and make the content a JSON object
stationJSON = json.loads(resp.decode('utf-8'))
# stationJSON

### Setting up our data is also a lot like the last example except for two things. 

We would like to only work with stations that meet our criteria and filter out stations which don't report both variables. We'll handle both of these cases at the same time to avoid redundancy

In [75]:
stations = []
for station in stationJSON['STATION']:
    try:
        if station['OBSERVATIONS']['wind_speed_value_1']['value'] > 15 and station['OBSERVATIONS']['relative_humidity_value_1']['value'] < 30:
            stations.append([station['STID'], station['LATITUDE'], station['LONGITUDE'], station['STATE'],
                 station['OBSERVATIONS']['wind_speed_value_1']['value'], 
                 station['OBSERVATIONS']['relative_humidity_value_1']['value'],
                 station['OBSERVATIONS']['relative_humidity_value_1']['date_time']])
    except KeyError:
        pass
# stations

### Great! With data in hand, let's build the body of our e-mail.

Building out our email is pretty straightforward. I add a little description and then each station is a new line in our email with the observed conditions. 

In [76]:
message = 'The below stations reported winds greater than 15 mph and relative humidity lower than 30% in the last hour. Times are local to the user. \r\n \r\n'
for station in stations:
    message += str(station[0]) + ', located in ' + station[3] + ' at ' + str(station[1]) + ',' + str(station[2]) + ', reports wind at ' + str(station[4]) + ' mph and RH at ' + str(station[5]) + '%. Observed time: ' + str(station[6]) + '\r\n'
message += '\r\n Love Always,\r\n \r\n Robot Emailer'
# message

### Now the fun part. Sending the e-mail. We'll need to import some built-in libraries to bring it all together.

I've went ahead and created an email address for us to use at fbfc.mwslapi@gmail.com. The password is studentFBFC. I'll delete this account after the conference is over so feel free to play around with it for right now. 

In [77]:
import smtplib
from email.mime.text import MIMEText
from time import strftime 

# define our content
recipients = ["fbfc.mwslapi@gmail.com"]
sender = "fbfc.mwslapi@gmail.com"
subject = strftime("%Y-%m-%d %H:%M:%S") + ' Fire Weather Stations'  # Try adding your name here to keep track

# make up the message
msg = MIMEText(message)  # this is the beautified list of stations
msg['Subject'] = subject
msg['From'] = sender
msg['To'] = ", ".join(recipients)  # Your recipients must always be a list

# sending
session = smtplib.SMTP('smtp.gmail.com', 587)
session.starttls()
session.login(sender, 'studentFBFC')
send_it = session.sendmail(sender, recipients, msg.as_string())
session.quit()

(221, b'2.0.0 closing connection m10sm13607387pfi.32 - gsmtp')

You should see an e-mail show up in the inbox of the fbfc.mwslapi@gmail.com account. Try sending an e-mail to yourself!